# Text Mining: Proyecto semestral

Juan Pablo Muñoz

martes, 15 de enero del 2019

## Introducción

En este *notebook* se resumen los experimentos y avances logrados en el desarrollo del proyecto semestral del curso *Text Mining* 2018-2. El proyecto tiene por objetivo aplicar los conocimientos adquiridos durante el curso para proponer una solución a algún problema dentro o fuera del área, con algún elemento novedoso.

Este proyecto propone una serie de métodos y algoritmos para la creación de vectores de palabras basados en Word2vec que no sufran del problema de la *fusión de significados*

La *fusión de significados* es una propiedad indeseada que se manifiesta en las representaciones vectoriales que se basan en Word2vec. En su formato original, Word2vec intenta generar un espacio vectorial en el que cada palabra observada pueda ser representada, de manera que, en el nuevo espacio, ésta quede cerca de las palabras que tienden a aparecer en su cercanía, a la vez que queda lejos de palabras que tienden a no aparecer cerca o en una misma frase. Pero este modelo de representación no considera dos propiedades muy importantes de la semántica: 1) que las palabras pueden tener varios significados y 2) que el significado de una palabra en uso depende fuertemente de las palabras que la rodean (contexto). Esta limitación del diseño de Word2vec causa que el modelo, por ejemplo, sólo produzca una única representación para la palabra "ratón", cuando en realidad "ratón" puede referirse a un animal mamífero o a un dispositivo tecnológico. Al mismo tiempo, el modelo se esforzará por hacer que todas las palabras fuertemente relacionadas a "ratón (animal)" y "ratón (tecnología)" queden cerca de la representación fusionada de "ratón", por lo que se deshace la propiedad distributiva del espacio vectorial resultante.


## Metodología de Trabajo

El proceso de creación de vectores Word2vec tiene las siguientes etapas:

 - Obtención de un corpus, que es un conjunto de documentos de texto
 - Preprocesar el corpus
 - Crear y entrenar un modelo Word2vec sobre el corpus preprocesado
 
Para poner en uso el modelo de representación creado, debe existir una consulta. Esta consulta es una cadena de texto, que debe ser preprocesada de la misma manera que el corpus antes de crear el modelo, para luego ser transformada al espacio de representación del modelo, donde se puede realizar la tarea que se requiera (por ejemplo, recuperación de información o resolución de preguntas, entre otros).
 
** La etapa más importante, costosa y complicada en este proyecto resulta ser el de preprocesamiento de texto**. Fue experimentando aquí donde se concentró todo el esfuerzo y tiempo del proyecto.

Luego de lograda esa parte, se puede entrenar un modelo word2vec para realizar *tests* de coherencia interna del espacio vectorial y para realizar *downstream tasks*. Sin embargo, estas actividades sólo sirven para el propósito de validación y no se les dio prioridad en esta iteración del trabajo.

## Datos

El corpus utilizado fue el depósito general de artículos de la Wikipedia en inglés: [enwiki-latest-pages-articles-multistream.xml.bz](https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles-multistream.xml.bz2) (tamaño ~15 GB con ~4.5 millones de artículos). Para los experimentos realizados, se utilizó una muestra aleatoria de un ~10% de los artículos.

Los métodos y algoritmos desarrollados utilizan fuentes de conocimiento léxico externas al corpus de texto utilizado para entrenar. En esta iteración del trabajo, se utiliza WordNet y la interfaz de NLTK para tener acceso simplificado a las estructuras de datos léxicas y a las operaciones realizables sobre ellas.

## Descripción del Preprocesado

Nota: Debido al alto costo computacional que tiene (de momento) decidir a priori si una palabra es o no ambigua, y la desambiguación implementada, se requiere de la indicación explícita de aquellas palabras a desambiguar durante el preprocesado del corpus.

A continuación, se describe el preprocesado del corpus, dada una lista de palabras ambiguas. Los nombres y argumentos descritos aquí no corresponden a los implementados. Esta modificación se hizo para priorizar rapidez de escritura y facilidad de entendimiento. La descripción detallada de las funciones se encuentra en cada implementación.

```
def preprocesado(corpus, lista_palabras_ambiguas):
  corpus_preprocesado = list()
  for documento in corpus:
    tokenizar(documento)
    remover_stopwords(documento)
    lematizar(documento)
    hacer_part_of_speech_tag(documento)
    desambiguar(documento, lista_palabras_ambiguas)
    corpus_preprocesado.append(documento)
  return corpus_preprocesado
```

Del anterior proceso, es necesario explicar `desambiguar()`:

```
def desambiguar(documento, lista_palabras_ambiguas):
  for palabra in documento:
    if palabra in lista_palabras_ambiguas:
      contexto = ventana(documento, palabra)
      palabra_desambiguada = decidir_significado(palabra, contexto)
      reemplazar(documento, palabra, palabra_desambiguada)
    return documento
```

Donde, `decidir_significado()` es una función que accede a los diccionarios de WordNet y permite decidir el significado más probable de una palabra dado su contexto, en base a múltiples métricas de similitud definidas sobre las estructuras de datos de WordNet llamadas `Synsets`, que se comportan como grafos y son utilizadas para representar relaciones jerárquicas entre términos relacionados, definiciones, ejemplos, etc.

## Código

### Clase y métodos para cargar el corpus de Wikipedia haciendo primeros pasos de preprocesado

Cargar el archivo `bz2` del corpus de la Wikipedia con una clase especialmente diseñada en la librería **gensim**. Esta clase permite realizar tokenización, transformación a letras minúsculas, remoción de *stop words*, lematización y *part-of-speech tagging* al momento de la carga de cada artículo. De manera adicional, se implementa y aplica sobre cada artículo una función normalizadora `remove_accents()`, que transforma los carácteres alfabéticos tildados a su forma original (ej.: "árbol"->"arbol").

In [0]:
from __future__ import print_function

import time
import logging
import os.path
import sys
import random

import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')


import nltk
from nltk.corpus import wordnet

lmtzr = nltk.WordNetLemmatizer().lemmatize

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


def lemmatize_text(text):
    word_pos = nltk.pos_tag(nltk.word_tokenize(text))
    lemm_words = [lmtzr(sw[0], get_wordnet_pos(sw[1])) for sw in word_pos]

    return [x.lower() for x in lemm_words]

'''
class gensim.corpora.wikicorpus.WikiCorpus(
    fname,
    processes=None, 
    lemmatize=True, 
    dictionary=None, 
    filter_namespaces=('0', ), 
    tokenizer_func=<function tokenize>, 
    article_min_tokens=50, 
    token_min_len=2, 
    token_max_len=15, 
    lower=True, 
    filter_articles=None
)

'''

from gensim.corpora import WikiCorpus

class WikiCorpusLoader:

    def __init__(self, wiki_dump_file, just_lemmatize=False, pos=False, dictionary={}):
        self.wiki = WikiCorpus(wiki_dump_file, lemmatize=pos, dictionary=dictionary)
        program = os.path.basename(sys.argv[0])
        self.logger = logging.getLogger(program)
        self.lemmatize = just_lemmatize
        self.pos = pos
        logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
        logging.root.setLevel(level=logging.INFO)
        self.logger.info("running %s" % ' '.join(sys.argv))
        
        
    def generate_sample(self, output_file, sample_frac=0.01, random_state_seed=99):
        assert sample_frac > 0 and sample_frac <= 1
        #assert not os.path.exists(output_file)
        output_no_lemma = open(output_file+'_no_lemma.txt', 'w')
        if self.lemmatize:
            output_lemma = open(output_file+'_lemma.txt', 'w')
        if self.pos:
            output_pos = open(output_file+'_pos.txt', 'w')
        i = 0
        idx = 0
        period = 100000
        if self.pos:
            period = 2000
        random.seed(random_state_seed)
        for article in self.wiki.get_texts():
            idx += 1
            if sample_frac < 1:
                article_selected = random.random() <= sample_frac
                if not article_selected:
                    continue
            else:
                i = i + 1
                if not self.pos:
                    normalized_article = bytes(' '.join(remove_accents(w) for w in article), 'utf-8').decode('utf-8')
                else:
                    normalized_article = ' '.join(remove_accents(w.decode('utf-8')) for w in article)
                output_no_lemma.write(str(idx)+':'+normalized_article+'\n')
                if self.lemmatize:
                    output_lemma.write(str(idx)+':'+' '.join(lemmatize_text(normalized_article))+'\n')
                if self.pos:
                    output_pos.write(str(idx)+'\n')
                
                if (i % period == 0):
                    self.logger.info("Saved " + str(i) + " articles")
        output_no_lemma.close()
        if self.lemmatize:
            output_lemma.close()
        if self.pos:
            output_pos.close()
        self.logger.info("Finished saving " + str(i) + " articles.")

### Funciones para la desambiguación de un corpus preprocesado

Estas funciones permiten iterar sobre los documentos preprocesados, realizando desambiguación automática de todas las palabras que se hayan indicado. Luego de este paso, el corpus queda listo para ser usado en el entrenamiento de modelos Word2vec.

In [55]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

def get_similarity(ss1, ss2, method='path'):
  if method =='path':
    return max(
        s for s in [
            0,
            wn.path_similarity(ss1,ss2,simulate_root=False),
            wn.path_similarity(ss2,ss1,simulate_root=False),
        ] if s is not None
    )
  if method =='wup':
    return max(
        s for s in [
            0,
            wn.wup_similarity(ss1,ss2,simulate_root=False),
            wn.wup_similarity(ss2,ss1,simulate_root=False),
        ] if s is not None
    )


def perform_deambiguation(
    document,
    target_word_index,
    method='path',
    verbose=False,
):
  
  '''
  deambiguates a word given a context, by comparing all WordNet synsets of the
  target word with those of the words in the context
  word: 2-tuple (<token>, <pos_tag>)
  context: list of 2-tuples [(<token>, <pos_tag>), ...]
  
  Method: Choose the meaning that maximizes the sum of similarities between it
  and the most similar sense of every word in the context.
  
  Intuitively: Select the meaning that "makes the more sense" possible given the
  most convenient interpretation of every word in the context.
  
  Note: currently using path-based similarity -> Higher similarity when score is
  higher.
  '''
  
  assert target_word_index < len(document) and 0 <= target_word_index
  
  word = document[target_word_index]
  context_pre = document[0:target_word_index]
  context_post = document[target_word_index+1:]
  context = context_pre+context_post
  
  # Base case: if the target word is not in WordNet, return the document
  # unmodified
  candidate_meanings = wn.synsets(word[0], pos=word[1])
  if not candidate_meanings:
    return list(w[0] for w in document)
  if verbose:
    print('Context:',context)
    for candidate_meaning in candidate_meanings:
      print('Target:',candidate_meaning.name())
      for context_word, context_pos in context:
        for context_word_synset in wn.synsets(context_word, pos=context_pos):
          print('similarity({}, {})=\t\t\t{}'.format(
              candidate_meaning.name(),
              context_word_synset.name(),
              get_similarity(candidate_meaning, context_word_synset, method)
          ))

  scores = dict()
  # Iterate over all possible meanings of the word to be deambiguated
  for candidate_meaning in wn.synsets(word[0], pos=word[1]):
    
    # Evaluate "how much sense" makes the candidate meaning with the context
    similarities = ([
        get_similarity(candidate_meaning, context_word_synset, method) \
        for context_word_synset in wn.synsets(context_word, pos=context_pos)
    ] for context_word, context_pos in context)
    scores[candidate_meaning] = sum(max(s) for s in similarities)
  
  # Choose the meaning that makes "more sense"
  result = max(scores, key=scores.get)
  
  # Strip PoS tags off since they are no longer needed
  tagless_context_pre = []
  tagless_context_post = []
  if len(context_pre) > 0:
    tagless_context_pre = [w for w, tag in context_pre]
  if len(context_post) > 0:
    tagless_context_post = [w for w, tag in context_post]
    
  # Reconstruct document with the deambiguated target word
  result_doc = tagless_context_pre+[result.name()]+tagless_context_post
  return result_doc

def penn_to_wn(pos_tag):
    if pos_tag.startswith('J'):
        return wn.ADJ
    elif pos_tag.startswith('N'):
        return wn.NOUN
    elif pos_tag.startswith('R'):
        return wn.ADV
    elif pos_tag.startswith('V'):
        return wn.VERB
    return None

def penn_to_wn_document(doc):
  wn_doc = []
  for token, pos_tag in doc:
    wn_doc.append((token, penn_to_wn(pos_tag)))
  return wn_doc
  
def pattern_to_wn_pos_transform(document):
  
  '''
  ['token_1/POS_1', ...] -> [('token_1', 'POS_1'), ...]
  '''
  
  r = [(t.split('/')[0], t.split('/')[1]) for t in document]
  return r
  
def deambiguate_article(
    document, 
    word_list, 
    pos_tagged=False, 
    method='path',
    window_size=2,
    verbose=False,
):
  
  '''
  Processes a document, first by stripping any stop word off it, then by
  performing a PoS tagging for specific words to be deambiguated and their 
  contexts, of size window_size. After the PoS tagging, tries to deambiguate the
  target word.
  
  document: list of word strings
  word_list: words to be PoS tagged
  pos_tagged: True if document comes already PoS tagged (format: 'token/POS')
  method: 'path' for path-based similarity measure. 'ic' for 
  information-content-based similarity measure (TODO: support for 'ic')
  window_size: number of words before and after a target word to be considered
  as its context. Ideally, a bigger window should improve the quality of the PoS
  tagging and deambiguation until certain point (too big windows sizes could 
  span more than one context, phrase or paragraph).
  '''
  if not pos_tagged:
    stopwords_eng = stopwords.words('english')
    document = [token for token in document if token not in stopwords_eng]
  deambiguated_doc = ['']*len(document)
  for index, w in enumerate(document):
    if deambiguated_doc[index] != '':
      continue
    tagless_w = w.split('/')[0]
    deambiguated_doc[index] = tagless_w
    if tagless_w in word_list:
      window_start = max(0, index-window_size)
      window_end = min(len(document), index+window_size+1)
      target_w_index = index-window_start
      window = document[window_start:window_end]
      if pos_tagged:
        pos_tagged_window = pattern_to_wn_pos_transform(window)
        pos_tagged_window = penn_to_wn_document(pos_tagged_window)
      else:
        pos_tagged_window = penn_to_wn_document(nltk.pos_tag(window))
      deambiguated_w = perform_deambiguation(
          pos_tagged_window, 
          target_w_index,
          method=method,
          verbose=verbose,
      )
      deambiguated_doc[window_start:window_end] = deambiguated_w
  return deambiguated_doc

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Lectura del corpus

In [0]:
# Lectura de corpus

%%time

print("Cargando y procesando corpus de Wikipedia...")
corpus_loader = WikiCorpusLoader(
    wiki_dump_file='data/enwiki-latest-pages-articles-multistream.xml.bz2',
    just_lemmatize=False,
    pos=True
)

corpus_loader.generate_sample(
    output_file='data/wikicorpus_0.1',
    sample_frac=0.1,
)

**Salida de la celda de lectura de corpus**:

2019-01-09 19:57:56,101: INFO: running c:\program files\python36\lib\site-packages\ipykernel_launcher.py -f C:\Users\Juampiblo\AppData\Roaming\jupyter\runtime\kernel-baa5de1c-3e6f-46dc-8875-cd40d4c3abe0.json

Cargando muestra del corpus de Wikipedia...

2019-01-09 20:02:45,123: INFO: Saved 1000 articles

2019-01-09 20:06:29,344: INFO: Saved 2000 articles

2019-01-09 20:09:38,782: INFO: Saved 3000 articles

...

2019-01-10 11:44:28,385: INFO: Saved 456000 articles

2019-01-10 11:45:24,977: INFO: Saved 457000 articles

2019-01-10 11:46:24,737: INFO: Saved 458000 articles

2019-01-10 11:47:15,700: INFO: finished iterating over Wikipedia corpus of 
4583951 documents with 2556203780 positions (total 19096287 articles, 
2624561277 positions before pruning articles shorter than 50 words)
2019-01-10 11:47:16,063: INFO: Finished saving 458949 articles.
Wall time: 15h 49min 19s

### Muestra del corpus cargado

Luego de la lectura del corpus, se produce un archivo de texto plano que contiene un artículo preprocesado por línea. Si se lee un artículo en una lista dividida por espacios (`split()`), el formato de cada artículo es el siguiente:

In [73]:
# Muestra de uno de los artículos del corpus y su formato final

a_sub = [b'anarchism/NN', b'be/VB', b'political/JJ', b'philosophy/NN', b'advocate/VB', b'self/NN', b'govern/VB', b'society/NN', b'base/VB', b'voluntary/JJ', b'cooperative/JJ', b'institution/NN', b'reject/VB', b'unjust/JJ', b'hierarchy/NN', b'institution/NN', b'be/VB', b'often/RB', b'describe/VB', b'stateless/JJ', b'society/NN', b'several/JJ', b'author/NN', b'have/VB', b'define/VB', b'more/RB', b'specifically/RB', b'institution/NN', b'base/VB', b'hierarchical/JJ', b'free/JJ', b'association/NN', b'anarchism/NN', b'hold/VB', b'capitalism/NN', b'state/NN']
a_sub = [w.decode('utf-8') for w in a_sub]
a_sub

['anarchism/NN',
 'be/VB',
 'political/JJ',
 'philosophy/NN',
 'advocate/VB',
 'self/NN',
 'govern/VB',
 'society/NN',
 'base/VB',
 'voluntary/JJ',
 'cooperative/JJ',
 'institution/NN',
 'reject/VB',
 'unjust/JJ',
 'hierarchy/NN',
 'institution/NN',
 'be/VB',
 'often/RB',
 'describe/VB',
 'stateless/JJ',
 'society/NN',
 'several/JJ',
 'author/NN',
 'have/VB',
 'define/VB',
 'more/RB',
 'specifically/RB',
 'institution/NN',
 'base/VB',
 'hierarchical/JJ',
 'free/JJ',
 'association/NN',
 'anarchism/NN',
 'hold/VB',
 'capitalism/NN',
 'state/NN']

### Desambiguación

A continuación, se realiza el último paso del preprocesado, que es la desambiguación.

Para desambiguar una palabra, se debe

 - Entregar un contexto (lista de palabras) que acompaña a la palabra a desambiguar. Recordar que el sentido de una palabra está fuertemente relacionado a las palabras de su contexto.
 - Entregar lematizados y PoS-taggeados tanto la palabra como el contexto. La lematización es un paso necesario para llevar todas las variaciones o conjugaciones de las palabras a su forma base, y permite identificarlas independientemente cómo se usen. El *PoS-tagging* permite a los métodos desamgibuadores reducir la cantidad inicial de significados posibles de una palabra ambigua. Si el *PoS-tagger* ha identificado a la palabra "light" como un adjetivo dado el contexto en el que se observó, entonces los métodos desambiguadores desambiguarán solamente considerando los significados de "light" que sean adjetivos entre los candidatos.
 
 La salida de la siguiente celda muestra el proceso comparativo que realizan los métodos desambiguadores sobre todos los significados candidatos de cada palabra ambigua (presente en la lista de palabras ambiguas `word_list`) encontrada.
 
Cuando se encuentra una palabra a desambiguar en el artículo, se cuantifica "cuánto sentido" tiene cada posible significado de la misma con las palabras de su contexto. Para esto, se comparan los `synsets` del significado candidato con todos los `synsets` de todas las palabras del contexto. Con `n` significados candidatos, dado un contexto de `m` palabras con `o` `synsets` cada una, la cantidad de veces que se aplica la medida de similitud entre `synsets` está en el orden `O(n*m*o)`.

In [69]:
%%time

a_sub_deambiguated = deambiguate_article(
    document=a_sub, 
    word_list=['institution', 'govern', 'have', 'define'], 
    pos_tagged=True, 
    window_size=2, 
    verbose=True
)

Context: [('advocate', 'v'), ('self', 'n'), ('society', 'n'), ('base', 'v')]
Target: regulate.v.02
similarity(regulate.v.02, recommend.v.01)=			0
similarity(regulate.v.02, preach.v.02)=			0
similarity(regulate.v.02, self.n.01)=			0
similarity(regulate.v.02, self.n.02)=			0
similarity(regulate.v.02, society.n.01)=			0
similarity(regulate.v.02, club.n.02)=			0
similarity(regulate.v.02, company.n.03)=			0
similarity(regulate.v.02, society.n.04)=			0
similarity(regulate.v.02, establish.v.08)=			0
similarity(regulate.v.02, base.v.02)=			0
similarity(regulate.v.02, free-base.v.01)=			0
Target: govern.v.02
similarity(govern.v.02, recommend.v.01)=			0
similarity(govern.v.02, preach.v.02)=			0
similarity(govern.v.02, self.n.01)=			0
similarity(govern.v.02, self.n.02)=			0
similarity(govern.v.02, society.n.01)=			0
similarity(govern.v.02, club.n.02)=			0
similarity(govern.v.02, company.n.03)=			0
similarity(govern.v.02, society.n.04)=			0
similarity(govern.v.02, establish.v.08)=			0
similarity(g

In [70]:
a_sub_deambiguated

['anarchism',
 'be',
 'political',
 'philosophy',
 'advocate',
 'self',
 'regulate.v.02',
 'society',
 'base',
 'voluntary',
 'cooperative',
 'institution.n.01',
 'reject',
 'unjust',
 'hierarchy',
 'institution.n.01',
 'be',
 'often',
 'describe',
 'stateless',
 'society',
 'several',
 'author',
 'have.v.12',
 'define',
 'more',
 'specifically',
 'institution.n.01',
 'base',
 'hierarchical',
 'free',
 'association',
 'anarchism',
 'hold',
 'capitalism',
 'state']

### Comparación normal vs. desambiguado

Se puede comparar la muestra del artículo previo y posterior a la desambiguación:

In [78]:
print("ANTES | DESPUÉS")
for w, x in zip(a_sub, a_sub_deambiguated):
  print(w.split('/')[0], '|', x)

ANTES | DESPUÉS
anarchism | anarchism
be | be
political | political
philosophy | philosophy
advocate | advocate
self | self
govern | regulate.v.02
society | society
base | base
voluntary | voluntary
cooperative | cooperative
institution | institution.n.01
reject | reject
unjust | unjust
hierarchy | hierarchy
institution | institution.n.01
be | be
often | often
describe | describe
stateless | stateless
society | society
several | several
author | author
have | have.v.12
define | define
more | more
specifically | specifically
institution | institution.n.01
base | base
hierarchical | hierarchical
free | free
association | association
anarchism | anarchism
hold | hold
capitalism | capitalism
state | state


## Conclusión

El costo de desambiguar palabras es alto, pero se puede reducir en gran medida realizando preprocesado de manera *offline*: tokenización, remoción de *stop words*, lematización y *PoS-tagging*.

Es necesario que el modelo preprocesador pueda determinar autónomamente aquellas palabras a desambiguar, pero en esta iteración no se logró ni fue una gran prioridad. Lo será durante las siguientes iteraciones, donde los casos de prueba tendrán mayor tamaño, y donde es necesaria la independencia de los modelos con respecto al *input* manual de humanos.

Si bien el trabajo desarrollado en esta iteración no alcanzó a cubrir las tareas de validación, como el entrenamiento de modelos Word2vec basados en el corpus desambiguado, y las *downstream tasks* como recuperación de información, se sostiene que éstas pueden ser razonablemente postergadas en favor de lograr refinaciones en el proceso de desambiguación (tareas que consumen tiempo, pues implican la carga y procesamiento de grandes volúmenes de información).

